In [8]:
from pathlib import Path
from tqdm.auto import tqdm

import pandas as pd
import hvplot.pandas

import torch
import xtensor as xt

import diffhydro as dh
import diffhydro.pipelines as dhp

from utils.io import load_vpu

### Parameters

In [2]:
plength_thr=10**4
node_thr=10**4
max_delay = 30
dt = 1/24

device = "cuda:7"
tr_vpu = "603"
te_vpu = "602"

n_iter_per_cluster = 100
n_clusters = 20
n_iter=20
n_epoch=20

init_len, pred_len = max_delay, 1024

In [3]:
root = Path("./data")
root = root / "geoglows"
interp_path = root / "interp_weight.feather"
runoff_path = root / "daily_sparse_runoff.feather"

### Data Loading

In [4]:
interp_df = pd.read_feather(interp_path).set_index("river_id")
runoff = xt.read_feather(runoff_path, dims=["time", "spatial"])\
            .sel(time=slice(None, "2019")).to(device)\
            .expand_dims("batch")\
            .transpose("batch", "spatial", "time") / (3600. * 24)

tr_g, tr_discharge, tr_runoff = load_vpu(root, tr_vpu, runoff, interp_df, device="cpu",
                                         plength_thr=plength_thr, node_thr=node_thr)
te_g, te_discharge, te_runoff = load_vpu(root, te_vpu, runoff, interp_df, device="cpu",
                                         plength_thr=plength_thr, node_thr=node_thr)

#### Upstream stats computations ... ####


Computing breakpoints:   0%|          | 0/55576 [00:00<?, ?it/s]

#### Segmentation into subgraphs ... ####
Removing edges...


  0%|          | 0/55576 [00:00<?, ?it/s]

Segment graph into connected components....
Build subgraphs for each cluster and node-cluster map...


...

  0%|          | 0/17 [00:00<?, ?it/s]

Computing merged graphs node idxs...


  0%|          | 0/17 [00:00<?, ?it/s]

Match breakpoint nodes across clusters...


  0%|          | 0/17 [00:00<?, ?it/s]

In [5]:
tr_data = dhp.BaseDataset(x=tr_runoff, y=tr_discharge, g=tr_g.to(device),
                          init_len=init_len, pred_len=pred_len)
te_data = dhp.BaseDataset(x=te_runoff, y=te_discharge, g=te_g.to(device),
                          init_len=init_len, pred_len=pred_len)

### Moderate-size-graph routing parameter learning

In [14]:
param_model = dhp.MLP(2,2)
model  = dhp.LearnedRouter(irf_name="muskingum",
                           param_model=param_model,
                           temp_res_h=24,
                           max_delay=max_delay, 
                           dt=dt).to(device)
module = dhp.RoutingModule(model, tr_data, te_data, 
                           device=device,
                           batch_size=1, lr=.001)#.to(device)

te_nse, tr_nse = module.train(n_iter=n_iter, n_epoch=n_epoch)

(te_nse.hvplot(label="Test") *\
 tr_nse.hvplot(label="Train")).opts(ylim=(.99, 1))

0it [00:00, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

:Overlay
   .Curve.Test  :Curve   [index]   (0)
   .Curve.Train :Curve   [index]   (0)

### Handling very large graphs

In [ ]:
param_model = dhp.MLP(2,2)
model  = dhp.LearnedRouter(irf_name="muskingum",
                           param_model=param_model,
                           temp_res_h=24,
                           max_delay=max_delay, 
                           dt=dt).to(device)
module = dhp.RoutingModule(model, tr_data, te_data, 
                           device=device,
                           batch_size=1, lr=.001)#.to(device)

results = module.train_sequentially(n_iter=n_iter, n_epoch=n_epoch)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]